In [ ]:
#---------------------------------------------------------------------------
#Step 1 : Data Import for Company CIN for financial year 2019-20                    
#---------------------------------------------------------------------------

In [ ]:
import pandas as pd
fy_19_20 =pd.read_excel(r"C:\Users\saura\Desktop\CSR\Data From National CSR Portal (Dec 2021)\CSR - Company CIN.xlsx","Financial Year 2019-2020", header=0) 
fy_19_20.count()

In [ ]:
fy_19_20_cin = fy_19_20.iloc[:11000,]
fy_19_20_cin.count()

In [ ]:
#22531 CIN
fy_cin_unique = fy_19_20_cin.drop_duplicates(keep='first',inplace=False)
fy_cin_unique.count()

In [ ]:
fy_cin_unique_new=fy_cin_unique["CIN"].values.tolist()
#print(fy_cin_unique_new)

In [ ]:
#-----------------------------------------------------------------------------------
#Step 2 : Web Scraping for variables from Table 1:
#-----------------------------------------------------------------------------------

In [ ]:
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')
import sys
import time

In [ ]:
ds_new=pd.DataFrame()
count = 0
retries = 1
for cin in fy_cin_unique_new:
    try:
        url = "https://www.csr.gov.in/companyprofile.php?year=FY+2019-20&CIN="+cin
        r = requests.get(url)
        #print(r,url)
        data = r.text
        soup = BeautifulSoup(data)
        count=count+1
        print(count)
        
        table = soup.find_all('table',class_="table")[0]
        #print(table)
        rows = table.find_all('tr')[1:]
        #print(rows)
        #Company Names
        header2 = soup.find_all('h2')[0]
        cname = header2.find_all('b')[0]
        cname = cname.text.strip()
        #print(count,cname)
        #Generate lists
        AA =[]
        BB =[]
    
        for row in table.find_all("tr"):
            states=row.findAll('th') #To store second column data
            cells = row.find_all('td')
            #length=len(cells)
            #print(length)
            if len(cells)==1: #Only extract table body not heading
                AA.append(states[0].find(text=True))
                BB.append(cells[0].find(text=True))
    
            df = pd.DataFrame(AA,columns=['Column1'])
            df['Column2']=BB
            df['CIN'] = cin
            df['CNAME'] = cname
            df.set_index('Column1', inplace = True)
            #print(df)
            f1 = [ds_new,df]
            ds_new = pd.concat(f1)
    except Exception as e:
        wait = retries * 30
        sys.stdout.flush()
        time.sleep(wait)
        retries += 1

In [ ]:
ds_new.head(10)

In [ ]:
ds_new.count()

In [ ]:
ds_new = ds_new.drop_duplicates(keep='first',inplace=False)
ds_new.reset_index(inplace =True)
ds_new.head(6)

In [ ]:
#Removing duplicates
Table_new = ds_new.drop_duplicates(keep='first',inplace=False)
Table_new.count()

In [ ]:
#-----------------------------------------------------------------------------------
#Step 5 : Transpose the data with pivot
#-----------------------------------------------------------------------------------
Table_final1 = Table_new.pivot(index=['CIN','CNAME'], columns='Column1', values='Column2')
Table_final1.head()

In [ ]:
Table_final1.to_csv(r"C:\Users\saura\Desktop\CSR\Data From National CSR Portal (Dec 2021)\All Data\Table 0 (FY 2019-20)\Table0_2019_20_one.csv")

### First Remaining CIN(s) : Missed CIN(s) due to connection issues with the website.

In [ ]:
#Unique CIN
Table_final1 = Table_final1.reset_index()
CIN = list(Table_final1.CIN.unique())

In [ ]:
n1= fy_19_20[~fy_19_20.CIN.isin(CIN)]

In [ ]:
print(n1,type(n1))

In [ ]:
fy_19_20_new=n1["CIN"].values.tolist()

In [ ]:
ds_new=pd.DataFrame()
count = 0
retries = 1
for cin in fy_19_20_new:
    try:
        url = "https://www.csr.gov.in/companyprofile.php?year=FY+2019-20&CIN="+cin
        r = requests.get(url)
        #print(r,url)
        data = r.text
        soup = BeautifulSoup(data)
        count=count+1
        print(count)
        
        table = soup.find_all('table',class_="table")[0]
        #print(table)
        rows = table.find_all('tr')[1:]
        #print(rows)
        #Company Names
        header2 = soup.find_all('h2')[0]
        cname = header2.find_all('b')[0]
        cname = cname.text.strip()
        #print(count,cname)
        #Generate lists
        AA =[]
        BB =[]
    
        for row in table.find_all("tr"):
            states=row.findAll('th') #To store second column data
            cells = row.find_all('td')
            #length=len(cells)
            #print(length)
            if len(cells)==1: #Only extract table body not heading
                AA.append(states[0].find(text=True))
                BB.append(cells[0].find(text=True))
    
            df = pd.DataFrame(AA,columns=['Column1'])
            df['Column2']=BB
            df['CIN'] = cin
            df['CNAME'] = cname
            df.set_index('Column1', inplace = True)
            #print(df)
            f1 = [ds_new,df]
            ds_new = pd.concat(f1)
    except Exception as e:
        wait = retries * 30
        sys.stdout.flush()
        time.sleep(wait)
        retries += 1

In [ ]:
ds_new.head(10)

In [ ]:
ds_new = ds_new.drop_duplicates(keep='first',inplace=False)
ds_new.reset_index(inplace =True)
ds_new.head(6)

In [ ]:
#Removing duplicates
Table_new = ds_new.drop_duplicates(keep='first',inplace=False)
Table_new.count()

In [ ]:
#-----------------------------------------------------------------------------------
#Step 6 : Transpose the data with pivot for remaining CIN
#-----------------------------------------------------------------------------------
Table_final2 = Table_new.pivot(index=['CIN','CNAME'], columns='Column1', values='Column2')
Table_final2.head()

In [ ]:
#Unique CIN
Table_final2 = Table_final2.reset_index()

In [ ]:
#-----------------------------------------------------------------------------------
#Step 4 : #Appending both tables
#-----------------------------------------------------------------------------------
Table_final = Table_final1.append(Table_final2,ignore_index=True)
Table_final.count()

In [ ]:
Table_final.to_csv(r"C:\Users\saura\Desktop\CSR\Data From National CSR Portal (Dec 2021)\All Data\Table 0 (FY 2019-20)\Table0_2019_20_two.csv")

### Second Remaining CIN(s) : Missed CIN(s) due to connection issues with the website.

In [1]:
import pandas as pd
fy_19_20 =pd.read_excel(r"C:\Users\saura\Desktop\CSR\Data From National CSR Portal (Dec 2021)\CSR - Company CIN.xlsx","Financial Year 2019-2020", header=0) 
fy_19_20.count()

Unnamed: 0    22531
CIN           22531
dtype: int64

In [2]:
Table_final = pd.read_csv(r"C:\Users\saura\Desktop\CSR\Data From National CSR Portal (Dec 2021)\All Data\Table 0 (FY 2019-20)\Table0_2019_20_two.csv")
Table_final.count()

Unnamed: 0        21768
CIN               21768
CNAME             21726
Class             21768
Company Type      21768
Listing Status    21768
RoC               21732
State             21768
Sub Category      21732
dtype: int64

In [3]:
CIN = list(Table_final.CIN.unique())
#print(CIN)

In [4]:
n1= fy_19_20[~fy_19_20.CIN.isin(CIN)]

In [5]:
print(n1,type(n1))

       Unnamed: 0                    CIN
167           167  L67100MH1995PLC093797
290           290  L65929TG2003PLC040648
365           365  L99999MH1996PLC132983
413           413  L74140MH2008PLC177884
651           651  L31904KA2019PLC121597
...           ...                    ...
22361       22361  U27109WB1991PTC053617
22409       22409  U65999RJ2018PTC063388
22417       22417  U72900DL2016PTC302660
22481       22481  U67120MH1999PTC281437
22507       22507  U67190WB1901PLC210002

[763 rows x 2 columns] <class 'pandas.core.frame.DataFrame'>


In [6]:
n1 = n1.drop_duplicates(keep='first',inplace=False)
n1.count()

Unnamed: 0    763
CIN           763
dtype: int64

In [7]:
fy_19_20_new=n1["CIN"].values.tolist()

In [8]:
print(fy_19_20_new, len(fy_19_20_new))

['L67100MH1995PLC093797', 'L65929TG2003PLC040648', 'L99999MH1996PLC132983', 'L74140MH2008PLC177884', 'L31904KA2019PLC121597', 'L74100TG1999PLC031191', 'L51909WB1982PLC034891', 'L65191TN1994PLC028734', 'L65191TN1992PLC022317', 'L65993DL1984PLC019141', 'L65100DL1993PLC053936', 'L67190DL1993PLC052050', 'L65920TG1923PLC000044', 'L67120WB1976PLC030625', 'L65923WB1982PLC035441', 'L51226WB1986PLC040287', 'L65993WB1983PLC036332', 'L17200MH1972PLC016008', 'L65990MH1984PLC032403', 'L67190MH1993PLC071117', 'L67120RJ1993PLC061489', 'L15205WB1944PLC011711', 'L65191TN1994PLC028333', 'L65929WB1990PLC048280', 'L51909WB1981PLC033699', 'L67120GJ1994PLC023803', 'L65910GJ1993PLC018956', 'L74910WB1994PLC064483', 'L51109WB1983PLC036091', 'L51909DL1984PLC019622', 'L01111WB1997PLC085164', 'L65993OR1927GOI034842', 'L74140DL1993PLC053485', 'L65923TG1986PLC006294', 'L65993WB1976PLC030622', 'L24231MH1994PLC334457', 'L65921WB1986PLC040980', 'L67120WB1972PLC028566', 'L51900MH1984PLC034709', 'L67120WB1991PLC052782',

In [9]:
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')
import sys
import time

In [10]:
ds_new=pd.DataFrame()
count = 0
retries = 1
for cin in fy_19_20_new:
    try:
        url = "https://www.csr.gov.in/companyprofile.php?year=FY+2019-20&CIN="+cin
        r = requests.get(url)
        #print(r,url)
        data = r.text
        soup = BeautifulSoup(data)
        count=count+1
        print(count)
        
        table = soup.find_all('table',class_="table")[0]
        #print(table)
        rows = table.find_all('tr')[1:]
        #print(rows)
        #Company Names
        header2 = soup.find_all('h2')[0]
        cname = header2.find_all('b')[0]
        cname = cname.text.strip()
        #print(count,cname)
        #Generate lists
        AA =[]
        BB =[]
    
        for row in table.find_all("tr"):
            states=row.findAll('th') #To store second column data
            cells = row.find_all('td')
            #length=len(cells)
            #print(length)
            if len(cells)==1: #Only extract table body not heading
                AA.append(states[0].find(text=True))
                BB.append(cells[0].find(text=True))
    
            df = pd.DataFrame(AA,columns=['Column1'])
            df['Column2']=BB
            df['CIN'] = cin
            df['CNAME'] = cname
            df.set_index('Column1', inplace = True)
            #print(df)
            f1 = [ds_new,df]
            ds_new = pd.concat(f1)
    except Exception as e:
        wait = retries * 30
        sys.stdout.flush()
        time.sleep(wait)
        retries += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
#ds_new = ds_new.drop_duplicates(keep='first',inplace=False)
ds_new.tail(6)

,Column2,CIN,CNAME
Column1,,,
Class,Private,U15311MH1940PTC003072,
State,Maharashtra,U15311MH1940PTC003072,
Company Type,Other than Govt. Companies,U15311MH1940PTC003072,
RoC,ROC�MUMBAI,U15311MH1940PTC003072,
Sub Category,Company limited by Shares,U15311MH1940PTC003072,
Listing Status,Unlisted,U15311MH1940PTC003072,


In [19]:
ds_new = ds_new.drop_duplicates(keep='first',inplace=False)
ds_new.reset_index(inplace =True)
ds_new.count()

Column1    54
Column2    54
CIN        54
CNAME      54
dtype: int64

In [20]:
#Removing duplicates
Table_new = ds_new.drop_duplicates(keep='first',inplace=False)
Table_new.count()

Column1    54
Column2    54
CIN        54
CNAME      54
dtype: int64

In [21]:
#-----------------------------------------------------------------------------------
#Step 6 : Transpose the data with pivot for remaining CIN
#-----------------------------------------------------------------------------------
Table_final3 = Table_new.pivot(index=['CIN','CNAME'], columns='Column1', values='Column2')
Table_final3.head()

,Column1,Class,Company Type,Listing Status,RoC,State,Sub Category
CIN,CNAME,,,,,,
U15311MH1940PTC003072,,Private,Other than Govt. Companies,Unlisted,ROC�MUMBAI,Maharashtra,Company limited by Shares
U17115AP2001PTC037792,Sri Nukala Rama Koteswara Rao Textiles Private Limited,Private,Other than Govt. Companies,Unlisted,0,Andhra Pradesh,Company limited by Shares
U24110MH1984PTC033855,Crown Chemicals Pvt Ltd,Private,Other than Govt. Companies,Unlisted,RoC-Mumbai,,Company limited by Shares
U34102TN2004PLC052489,Gulf Ashley Motor Limited,Public,Other than Govt. Companies,Unlisted,RoC-Chennai,,Company limited by Shares
U45200AP2015PTC096787,S V S Projects India Private Limited,Private,Other than Govt. Companies,Unlisted,0,Andhra Pradesh,Company limited by Shares


In [22]:
Table_final3.count()

Column1
Class             9
Company Type      9
Listing Status    9
RoC               9
State             9
Sub Category      9
dtype: int64

In [23]:
#Unique CIN
Table_final3 = Table_final3.reset_index()

In [24]:
#-----------------------------------------------------------------------------------
#Step 7 : #Appending both tables
#-----------------------------------------------------------------------------------
Table_final = Table_final.append(Table_final3,ignore_index=True)
Table_final.count()

Unnamed: 0        21768
CIN               21777
CNAME             21735
Class             21777
Company Type      21777
Listing Status    21777
RoC               21741
State             21777
Sub Category      21741
dtype: int64

In [25]:
CIN = list(Table_final.CIN.unique())
n1= fy_19_20[~fy_19_20.CIN.isin(CIN)]
n1.count()

Unnamed: 0    754
CIN           754
dtype: int64

In [26]:
#-----------------------------------------------------------------------------------
#Step 8 : #Appending ramining CIN with no profile view
#-----------------------------------------------------------------------------------
Table_final = Table_final.append(n1,ignore_index=True)
Table_final.count()

Unnamed: 0        22522
CIN               22531
CNAME             21735
Class             21777
Company Type      21777
Listing Status    21777
RoC               21741
State             21777
Sub Category      21741
dtype: int64

In [28]:
Table_final = Table_final[["CIN","CNAME","Class","Company Type","Listing Status","RoC","State","Sub Category"]]

In [29]:
Table_final.to_csv(r"C:\Users\saura\Desktop\CSR\Data From National CSR Portal (Dec 2021)\All Data\Table 0 (FY 2019-20)\Table0_2019_20_final.csv")